In [ ]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
import tensorflow as tf
import torch
from stylegan2_pytorch import StyleGAN2
def get_model(
    gpu,
    data='./data',
    results_dir='./results',
    models_dir='./models', 
    name='default',
    new=False,
    load_from=-1,
    image_size = 512,
    network_capacity = 16,
    transparent = False,
    batch_size = 1,
    gradient_accumulate_every = 1,
    num_train_steps = 150000,
    learning_rate = 1e-4,
    lr_mlp = 0.1,
    ttur_mult = 1.5,
    num_workers =  None,
    save_every = 1000,
    generate = False,
    generate_interpolation = False,
    save_frames = False,
    num_image_tiles = 8,
    trunc_psi = 0.75,
    fp16 = False,
    cl_reg = False,
    fq_layers = [],
    fq_dict_size = 256,
    attn_layers = [],
    use_feats=True,
    aug_prob = 0.,
    dataset_aug_prob = 0.,
):
    using_ddp = False
    model = StyleGAN2(
        image_size,        
        network_capacity = network_capacity,
        transparent = transparent,
        lr = learning_rate,
        lr_mlp = lr_mlp,
        ttur_mult = ttur_mult,
        fp16 = fp16,
        cl_reg = cl_reg,
        fq_layers = fq_layers,
        fq_dict_size = fq_dict_size,
        attn_layers = attn_layers,
        use_feats=use_feats,
        using_ddp=using_ddp,
        gpu=gpu
    )

    if using_ddp:
        world_size = torch.cuda.device_count()
        torch.distributed.init_process_group(backend='nccl', rank=gpu, world_size=world_size)
    else:
        gpu = 0
    torch.manual_seed(0)
    torch.cuda.set_device(gpu)
    load(model, models_dir, num=load_from)
    return model

In [ ]:
from pathlib import Path
def load(GAN, models_dir, dir_name='default', num=-1, gpu=0):
    name = num
    if num == -1:
        file_paths = [p for p in Path(models_dir + '/' + dir_name).glob('model_*.pt')]
        saved_nums = sorted(map(lambda x: int(x.stem.split('_')[1]), file_paths))
        if len(saved_nums) == 0:
            return
        name = saved_nums[-1]
        print(f'continuing from previous epoch - {name}')

    map_location = {'cuda:%d' % 0: 'cuda:%d' % gpu}
    print(str(models_dir + '/' + dir_name + '/' + f'model_{name}.pt'),)
    load_data = torch.load(str(models_dir + '/' + dir_name + '/' + f'model_{name}.pt'), map_location=map_location)
   # print(load_data['GAN'].keys())
    d2 = {}
    for k in load_data['GAN']:
      d2[k.replace('.module', '')] = load_data['GAN'][k]

    GAN.load_state_dict(d2)

In [ ]:
m = get_model(0)

In [ ]:
import torch
import numpy as np
torch.cuda.is_available()

In [ ]:
from helpers import noise, image_noise


def gen_styles(GAN, feats):
    torch.manual_seed(0)
    layers = GAN.num_layers
    n = noise(feats.shape[0], GAN.latent_dim)
    w_space = GAN.S(n, feats)
    styles = w_space[:,None,:].repeat(1,layers,1)
    return styles
    
w_styles = gen_styles(m, torch.zeros((1, 1024)).cuda())
n = image_noise(1, m.image_size)

In [ ]:
img = m.GE(w_styles, 0*n)

In [ ]:
%pylab inline

imshow(img.cpu().detach().numpy()[0].swapaxes(0,2).swapaxes(0,1))

In [ ]:
from xx.uncommon.column_store import ColumnStoreReader, ColumnStoreWriter
import random
from xx.uncommon.utils import transform_img

from common.transformations.camera import W as FULL_W                                                                                                                                                                                                                              
from common.transformations.camera import H as FULL_H                                                                                                                                                                                                                              
from common.transformations.model import MEDMODEL_INPUT_SIZE, medmodel_intrinsics
from tools.lib.framereader import FrameIterator
from common.transformations.camera import view_frame_from_device_frame as view_from_device


SEG_LIST = '/home/batman/openpilot/xx/pipeline/route_lists/1m_seg_list_500k.txt'
from xx.pipeline.require import convert_to_slash
with open(SEG_LIST) as f:
  segs3 = convert_to_slash(f.read().strip().split('\n'))
BASE_DIR = "http://data-ssd.comma.life/model_features/df05d416-a2c6-407e-8013-3c6c34e62d27_800/"

target_name = random.choice(segs3)
#target_name ='36e84c795c78116c|2019-07-17--15-53-07/6'

mf = ColumnStoreReader(os.path.join(BASE_DIR, "{}/{}".format('ModelFeatures', target_name)))
BASE_DIR = "http://data-ssd.comma.life/runner/training_1m/"

vt2 = ColumnStoreReader(os.path.join(BASE_DIR, "{}/{}".format('VisionTargets2', target_name)))
calib_view = view_from_device.dot(vt2['calib'][0])

In [ ]:
print(target_name)

In [ ]:
from tqdm import tqdm
import cv2

imgs = []
pbar = tqdm(enumerate(FrameIterator('cd:/'+target_name.replace("|" , "/") + "/fcamera.hevc", "rgb24")))                                                                                                                                                                                      
for fidx, rgb in pbar:
  img_o = transform_img(rgb, to_intr=medmodel_intrinsics,                                                                                                                                                                                         
                                 output_size=MEDMODEL_INPUT_SIZE,                                                                                                                                                                                                                  
                                 calib=calib_view, yuv=False)
  w_styles = gen_styles(m, torch.from_numpy(np.array(mf['values'][fidx:fidx+1])).cuda())
  img_g = m.G(w_styles, n)
  img_g = cv2.resize((np.clip(img_g.cpu().detach().numpy()[0]*256, 0 ,255)).astype(np.uint8).swapaxes(0,2).swapaxes(0,1)[256:], (512, 256))
  img = np.vstack((img_o, img_g))
  imgs.append(img)
imshow(img)

In [ ]:
figsize(12,12)
imshow(imgs[900])

In [ ]:
import cv2
import numpy as np
 

frameSize = (512, 512)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter('output_video.avi',fourcc, 20, frameSize)

for i in range(0,1199):
    img = imgs[i][:,:,::-1]
    out.write(img)

out.release()